#### Adult Dataset

In [41]:
import pandas as pd

# 定义数据集的列名
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
# 数据集的URL
url = "./adult.data"

# 使用pandas加载数据集
df = pd.read_csv(url, header=None, names=column_names, skipinitialspace=True)

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# 制定映射规则，全都转为数值型
# 只取age, education-num, capital-gain, capital-loss, hours-per-week列
input_X = df.iloc[:, [0, 4, 10, 11, 12]]
# input_X转为ndarray
input_X = input_X.values
# input_X
y = df['income']
df_y = y.str.strip().str.rstrip('.')
# 制定df_y的映射规则
# 假设你的目标类别是['<=50K', '>50K']
target_mapping = {
    '<=50K': 0,
    '>50K': 1,
}

input_y = pd.Series(df_y.map(target_mapping).values)
input_X = input_X[:1000]
input_y = input_y[:1000]
print("输入输出维度：", input_X.shape, input_y.shape)
print("Type: ", type(input_X), type(input_y))

X_train, X_test, y_train, y_test = train_test_split(input_X, input_y, test_size=0.2, random_state=42)

y_train = pd.factorize(df['income'])[0]  # 对y_train进行独热编码
y_test = pd.factorize(df_y)[0]  # 对y_test进行独热编码

X_train = torch.tensor(X_train, dtype=torch.float32)  # 转换为浮点数张量
y_train = torch.tensor(y_train, dtype=torch.float32)  # 转换为浮点数张量
X_test = torch.tensor(X_test, dtype=torch.float32)  # 转换为浮点数张量
y_test = torch.tensor(y_test, dtype=torch.float32)  # 转换为浮点数张量

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)  # 输入层到隐藏层，64个单元
        self.fc2 = nn.Linear(64, 32)  # 隐藏层到隐藏层，32个单元
        self.fc3 = nn.Linear(32, 16)  # 隐藏层到隐藏层，16个单元
        self.output = nn.Linear(16, 1)  # 隐藏层到输出层，1个单元（使用sigmoid激活函数）
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU激活函数
        x = torch.relu(self.fc2(x))  # ReLU激活函数
        x = torch.relu(self.fc3(x))  # ReLU激活函数
        output = self.output(x)  # sigmoid激活函数将输出限制在[0, 1]之间
        return output
    
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# 定义优化器和损失函数
model = NeuralNet()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCELoss()
num_epochs = 10

# 训练网络
for epoch in range(num_epochs):  # 你需要定义num_epochs
    for i, data in enumerate(X_train, 0):
        optimizer.zero_grad()
        output = model(data)  # this is your forward pass
        loss = criterion(output, y_train)  # compute loss
        loss.backward()  # compute gradients
        optimizer.step()  # update weights


输入输出维度： (1000, 5) (1000,)
0 tensor([49.,  9.,  0.,  0., 40.]) tensor(0.)
Output: tensor([-0.6384], grad_fn=<ViewBackward0>), label: 0.0
<class 'torch.Tensor'> <class 'torch.Tensor'>
Loss:  0.6383682489395142


AttributeError: 'float' object has no attribute 'backward'

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# 定义优化器和损失函数
model = NeuralNet()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCELoss()
num_epochs = 10

# 训练网络
for epoch in range(num_epochs):  # 你需要定义num_epochs
    for i, data in enumerate(X_train, 0):
        optimizer.zero_grad()
        output = model(data)  # this is your forward pass
        loss = criterion(output, y_train)  # compute loss
        loss.backward()  # compute gradients
        optimizer.step()  # update weights

ValueError: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.

In [ ]:
embedding = model.fc3.weight.data.cpu().numpy()